### Initialize an LLM

In [ ]:
# Import the AzureChatOpenAI class from the langchain_openai module
from langchain_openai import AzureChatOpenAI

# Import the load_dotenv and find_dotenv functions from the dotenv module
# These functions are used to load environment variables from a .env file
from dotenv import load_dotenv, find_dotenv

# Load the environment variables from the .env file, if it exists
load_dotenv(find_dotenv())

# Create an instance of the AzureChatOpenAI class,
# which represents a Large Language Model (LLM) from Azure OpenAI service.
# The model used is "gpt-4o" and it's using the 
# preview version of the API from March 2023.
# The 'verbose' parameter enables detailed logging for debugging.
# The 'temperature' parameter is set to 1, which controls the 
# randomness of the model's outputs (higher values mean more randomness).
llm = AzureChatOpenAI(model="gpt-4o",
                      api_version="2023-03-15-preview",
                      verbose=True,
                      temperature=0)

# Invoke the LLM with a prompt of "Hi" to generate a response.
llm.invoke("Hi")

### Asking deterministic questions

In [ ]:
llm.invoke("How many medals did USA win in the most recent Olympics?")

In [ ]:
llm.invoke("Who is Shanmukha Ranganath?")

In [ ]:
llm.invoke("How many 'm's are there in the word Weather")

In [ ]:
llm.invoke("""
Tell me something about the NBA Commissioner.
""")

### Sending Context along with question to enhance generation

In [ ]:
llm.invoke("""
The United States of America (USA), represented by the United States Olympic & Paralympic Committee (USOPC), 
competed at the 2024 Summer Olympics in Paris from July 26 to August 11, 2024. U.S. athletes have appeared in every 
Summer Olympic Games of the modern era, except for the 1980 edition in Moscow, when America led a sixty-six-nation 
boycott in protest of the Soviet invasion of Afghanistan. As Los Angeles is hosting the 2028 Summer Olympics, 
the United States marched penultimately before the homebound French team entered Place du Trocadéro during the 
parade of nations segment of the opening ceremony. Additionally, an American segment featuring H.E.R. and Tom Cruise from Paris, 
and the Red Hot Chili Peppers, Billie Eilish, Snoop Dogg, and Dr. Dre from Long Beach, was performed during the closing ceremony.
The opening ceremony flag-bearers for the United States were LeBron James and Coco Gauff. 
James, a two-time Olympic gold medalist, is the first male basketball player to be chosen.[1][2] 
At 20 years of age, Gauff is the youngest American athlete and the first tennis player to be so honored.[3][4]
The closing ceremony flag-bearers were Nick Mead and Katie Ledecky. 
Mead was part of the U.S. men's four rowing team that won their first gold medal since 1960, while Ledecky,
a nine-time Olympic gold medalist, became the most decorated female Olympian to be chosen.[5] 
The United States competed in all sports except handball and was represented by more female than male athletes for the 
fourth consecutive time in the Summer Olympics (278 men and 314 women).[6] 
The team topped the medal rankings for the fourth consecutive and 19th overall time, with a total of 40 gold, 44 silver, 
and 42 bronze medals.[7] Tied with China on golds (40), the U.S. placed first in the overall medal tally via a
tiebreaker (44–27 in silver). The occasion marked the first time in Summer Olympic history that two countries
finished with an equal number of gold medals at the top.[8] Additionally, the Americans won 126 medals overall
compared to China's 91.[9][10]

How many medals did USA win in the most recent Olympics?
""")

### Searching internet and sending top k (=3) elements to generation

In [ ]:
import requests

In [ ]:
import requests
import json
import os

url = "https://api.tavily.com/search"

payload = json.dumps({
  "query": "How many medails did USA won in 2024 Olympics",
  "api_key": os.getenv("TAVILY_API_KEY"),
  "search_depth": "basic",
  "topic": "general",
  "include_answer": False,
  "include_raw_content": False,
  "include_images": False,
  "include_image_descriptions": False,
  "include_domains": [],
  "max_results": 3
})

response = requests.request("POST", url, data=payload)

top_results = response.json()["results"]
print(top_results)
context = str.join("\n\n", [result['content'] for result in top_results])



llm.invoke(f"""
{context}

How many medals did USA win in the most recent Olympics?
""")

### Vector Embedding

In [ ]:
from langchain_community.document_loaders import YoutubeLoader

In [ ]:
from langchain_community.document_loaders.youtube import TranscriptFormat

loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=T-D1OfcDW1M",
    add_video_info=True,
    transcript_format=TranscriptFormat.CHUNKS,
    chunk_size_seconds=30,
)


In [ ]:
documents = loader.load()

In [ ]:
articles = [{'title': 'USA Olympic medal count 2024 - Sporting News', 'url': 'https://www.sportingnews.com/us/olympics/news/usa-best-olympic-medal-count-2024-paris-games-final-most-medals-all-time/0809c8a9c89cd4133bb0b281', 'content': 'The nation has reached at least 100 medals at an Olympics just 12 times, and at least 120 medals just four times. Yet the 2024 total is not the most medals Team USA has ever won.', 'score': 0.9728308, 'raw_content': None}, {'title': 'Olympic Games Paris 2024: All U.S. medal winners - full list', 'url': 'https://olympics.com/en/news/olympic-games-paris-2024-all-u-s-medal-winners-full-list', 'content': 'The United States arrived in the French capital with a contingent of nearly 600 athletes for the Olympic Games Paris 2024 from 26 July to 11 August.. Team USA came into the Games looking to exceed their haul from Tokyo 2020 in 2021 where it won 39 gold, 41 silver, and 33 bronze medals for a total of 113 medals. The North Americans have broken through the 100-medal mark at every Olympic Games ...', 'score': 0.95159405, 'raw_content': None}, {'title': 'Medal Count - Paris 2024 Olympic Medal Table', 'url': 'https://olympics.com/en/paris-2024/medals', 'content': "Official medal standings for the Paris 2024 Olympics (Jul 26-Aug 11, 2024). ... Site Language. En. Log in or Sign up. Back. Back. Medal Table. Medal Table Medallists # NOCs. G. S. B. USA United States of America. 40 44 42 126. CHN People's Republic of China. 40 27 24 91. JPN Japan. 20 12 13 45. AUS ... Does not include medals won by AIN. Timing ...", 'score': 0.92549974, 'raw_content': None}]

In [ ]:
article_embeddings = []

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings

In [ ]:
embedding_model = AzureOpenAIEmbeddings(model="text-embedding-ada-002-01",
                                 api_version="2023-05-15")

In [ ]:
for article in articles:
    embeddings = embedding_model.embed_query(article['content'])
    article_embeddings.append(embeddings)

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings

articles = [{'title': 'USA Olympic medal count 2024 - Sporting News', 'url': 'https://www.sportingnews.com/us/olympics/news/usa-best-olympic-medal-count-2024-paris-games-final-most-medals-all-time/0809c8a9c89cd4133bb0b281', 'content': 'The nation has reached at least 100 medals at an Olympics just 12 times, and at least 120 medals just four times. Yet the 2024 total is not the most medals Team USA has ever won.', 'score': 0.9728308, 'raw_content': None}, {'title': 'Olympic Games Paris 2024: All U.S. medal winners - full list', 'url': 'https://olympics.com/en/news/olympic-games-paris-2024-all-u-s-medal-winners-full-list', 'content': 'The United States arrived in the French capital with a contingent of nearly 600 athletes for the Olympic Games Paris 2024 from 26 July to 11 August.. Team USA came into the Games looking to exceed their haul from Tokyo 2020 in 2021 where it won 39 gold, 41 silver, and 33 bronze medals for a total of 113 medals. The North Americans have broken through the 100-medal mark at every Olympic Games ...', 'score': 0.95159405, 'raw_content': None}, {'title': 'Medal Count - Paris 2024 Olympic Medal Table', 'url': 'https://olympics.com/en/paris-2024/medals', 'content': "Official medal standings for the Paris 2024 Olympics (Jul 26-Aug 11, 2024). ... Site Language. En. Log in or Sign up. Back. Back. Medal Table. Medal Table Medallists # NOCs. G. S. B. USA United States of America. 40 44 42 126. CHN People's Republic of China. 40 27 24 91. JPN Japan. 20 12 13 45. AUS ... Does not include medals won by AIN. Timing ...", 'score': 0.92549974, 'raw_content': None}]

article_embeddings = []

embedding_model = AzureOpenAIEmbeddings(model="text-embedding-ada-002-01",
                                 api_version="2023-05-15")

for article in articles:
    embeddings = embedding_model.embed_query(article['content'])
    article_embeddings.append(embeddings)

In [ ]:
import numpy as np

In [ ]:
# Define a function to calculate cosine similarity between two vectors
def cosine_similarity(vec1, vec2):
    # Ensure both vectors are numpy arrays
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    # Compute cosine similarity
    cos_sim = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    return cos_sim

query = "Olympic medal count for USA in 2024"
query_embedding = embedding_model.embed_query(query)

similarities = []
for article, article_embedding in zip(articles, article_embeddings):
    similarity = cosine_similarity(query_embedding, article_embedding)
    similarities.append({
        'title': article['title'],
        'url': article['url'],
        'similarity': similarity
    })



In [ ]:
# Print the similarities
for result in similarities:
    print(f"Title: {result['title']}, Similarity: {result['similarity']:.4f}, URL: {result['url']}")